In [1]:
from typing import List, Tuple
from datetime import datetime, timedelta
import numpy.typing as npt
import numpy as np
import os
import cv2
import pickle
from tqdm import tqdm
import itertools
import json
import multiprocessing
import altair as alt
import pandas as pd

from utils.compose2_factory import compose2_factory
from utils.data_dirs import data_dirs
from utils.get_videos import get_videos
from utils.frame_json_encoder import FrameJSONEncoder
from utils.typings import Frame
from utils.load_images import load_images

In [2]:
raw_dir, processed_dir, folder, EXPERIMENT_DATA, suffix, *_ = data_dirs(False)
print(raw_dir)
print(processed_dir)
print(folder)
print(EXPERIMENT_DATA)
print(suffix)

/work/apperception/data/raw/nuScenes/full-dataset-v1.0/Mini
/data/apperception-data/processed/nuscenes/full-dataset-v1.0/Mini
v1.0-mini
/work/apperception/data/raw/scenic/experiment_data



In [3]:
fps = 20

In [4]:
interpolate = False

In [5]:
partition_dir = os.path.join(processed_dir, 'partition')

In [6]:
compose = compose2_factory(fps, raw_dir, processed_dir, interpolate)

In [7]:
def iframe(timestamp: "datetime"):
    return Frame(None, None, None, None, None, None, None, None, None, timestamp, None, None)

In [8]:
all_misalignments = []
for l in os.listdir(partition_dir):
    d = os.path.join(partition_dir, l)
    with open(os.path.join(d, 'sample_data.pickle'), "rb") as f:
        sample_data = pickle.load(f)
    videos = get_videos(sample_data)
    
    filenames = []
    for name, video in videos.items():
#         if 'scene-1077-CAM_BACK' not in name:
#             continue
        for f in video:
            filenames.append(f.filename)
    
    img_cache = load_images(raw_dir, filenames)
    print("Done Loading Images")
    
#     with multiprocessing.Pool() as pool:
#         inputs = ((l, name, video, img_cache, compose) for name, video in videos.items())
#         _outputs = tqdm(pool.imap_unordered(process_one_video, inputs), total=len(videos))
#         output = {
#             k: v
#             for k, v
#             in _outputs
#         }
    
    output = {}
    misalignments = []
    for name in videos:
#         if 'scene-1077-CAM_BACK' not in name:
#             continue
        video = videos[name]
        frames, filename, _misalignments = compose(l, name, video, img_cache)
        misalignments.extend(_misalignments)
#         prev = video[0][9]
#         for v in video[1:]:
#             print(int(round((v[9] - prev).total_seconds() * 100,)))
#             prev = v[9]
        o = {
            "filename": filename,
            "start": video[0][9],
            "columns": [
                "cameraId",
                "frameId",
                "frameNum",
                "filename",
                "cameraTranslation",
                "cameraRotation",
                "cameraIntrinsic",
                "egoTranslation",
                "egoRotation",
                "timestamp",
                "cameraHeading",
                "egoHeading",
            ],
            "frames": [
                (tuple(iframe(f)) if isinstance(f, datetime) else tuple(video[f]))
                for img, f in frames
            ]
        }
#         print('---------------------------')
        prev = o['frames'][0][9]
        for i, v in enumerate(o['frames'][1:30]):
#             print(int(round((v[9] - prev).total_seconds() * 100,)))
            assert int(round((v[9] - prev).total_seconds() * 100)) == 5, (int(round((v[9] - prev).total_seconds() * 100,)), i + 1, v)
            prev = v[9]
        output[name] = o
        print("done", filename)
    base = os.path.join(processed_dir, 'videos', l)
    with open(os.path.join(base, 'frames-skip.pickle'), "wb") as f:
        pickle.dump(output, f)
    
    with open(os.path.join(base, 'frames-skip.json'), "w") as f:
        json.dump(output, f, indent=2, sort_keys=True, cls=FrameJSONEncoder)
    
    # with open(os.path.join(base, 'misalignments.json'), "w") as f:
    #     json.dump([{'misalignment': m} for m in misalignments], f, indent=2)
    all_misalignments.extend([{'misalignment': m, "location": l} for m in misalignments])

Reading images


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4216/4216 [00:40<00:00, 104.31it/s]


Done Loading Images
misalignment: 0.0011814346 %
max misalignment: 0.088 %


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍| 236/237 [00:00<00:00, 162244.84it/s]


Writing scene (/data/apperception-data/processed/nuscenes/full-dataset-v1.0/Mini/videos/singapore-hollandvillage/skipped-scene-1077-CAM_FRONT_LEFT-2018-11-21-3-39-27-904844.mp4):


...

done skipped-scene-0061-CAM_BACK-2018-7-23-20-28-47-637525.mp4
misalignment: 0.0473972603 %
max misalignment: 5.652 %


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎| 218/219 [00:00<00:00, 290918.95it/s]


Writing scene (/data/apperception-data/processed/nuscenes/full-dataset-v1.0/Mini/videos/singapore-onenorth/skipped-scene-0061-CAM_BACK_LEFT-2018-7-23-20-28-47-647423.mp4):


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 384/384 [00:04<00:00, 77.04it/s]


done skipped-scene-0061-CAM_BACK_LEFT-2018-7-23-20-28-47-647423.mp4


In [9]:
# TODO: Partition data based on location
alt.Chart(pd.DataFrame(all_misalignments)).mark_tick().encode(
    alt.X('misalignment:Q', title='Misalignment (%)'),
    alt.Y('location:Q', title='Location'),
)

MaxRowsError: The number of rows in your dataset is greater than the maximum allowed (5000). For information on how to plot larger datasets in Altair, see the documentation

alt.Chart(...)